In [134]:
# basket of given stocks
!pip install requests
import pandas as pd
import requests
import numpy as np
from termcolor import colored as cl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [87]:
basket_of_Comps = ['MSFT', 'AMZN', 'GOOGL', 'FB', 'BABA', 'NVDA', 'PYPL', 'INTC', 'NFLX']
ticker = basket_of_Comps
tgt_ticker = 'AAPL' # Add target company here
ticker.append(tgt_ticker)
def get_metrics(stock):
    iex_api_key = 'sk_29735f4ddf4a47efb27623b229dda54a'
    fundamentals = []
    
    p = "https://cloud.iexapis.com/stable/stock/"+stock+"/price?token="+iex_api_key
    m = "https://cloud.iexapis.com/stable/stock/"+stock+"/stats?token="+iex_api_key
    pe = "https://cloud.iexapis.com/stable/stock/"+stock+"/stats?token="+iex_api_key
    ebitda = "https://cloud.iexapis.com/stable/time-series/fundamentals/"+stock+"/quarterly?token="+iex_api_key
    ebit = "https://cloud.iexapis.com/stable/time-series/fundamentals/"+stock+"/quarterly?token="+iex_api_key
    rev = "https://cloud.iexapis.com/stable/time-series/fundamentals/"+stock+"/quarterly?token="+iex_api_key
    ev = "https://cloud.iexapis.com/stable/stock/"+stock+"/advanced-stats?token="+iex_api_key

    raw_price = requests.get(p)
    price = raw_price.json()    
    fundamentals.append(price) 

    raw_marketcap = requests.get(m)
    marketcap = raw_marketcap.json()['marketcap']
    fundamentals.append(marketcap)    
 
    raw_peRatio = requests.get(pe)
    peRatio = raw_peRatio.json()['peRatio']
    fundamentals.append(peRatio)
    
    raw_ebitda = requests.get(ebitda)
    ebitda = raw_ebitda.json()[0]['ebitdaReported']
    fundamentals.append(ebitda)
        
    raw_ebit = requests.get(ebit)
    ebit = raw_ebit.json()[0]['ebitReported']
    fundamentals.append(ebit)
    
    raw_revenue = requests.get(rev)
    revenue = raw_revenue.json()[0]['revenue']
    fundamentals.append(revenue)
    
    raw_entvalue = requests.get(ev)
    entvalue = raw_entvalue.json()['enterpriseValue']
    fundamentals.append(entvalue)
    
      
    return fundamentals

raw_data = []

for company in ticker:
  raw_data.append(get_metrics(company))


In [88]:
fundamentals = pd.DataFrame(raw_data, columns = ['price', 'marketcap', 'pe', 'ebitda', 'ebit', 'revenue', 'ev'], index = ticker)


In [90]:
valuation_multiples = fundamentals[['pe',	'ebitda',	'ebit',	'revenue']].copy()
valuation_multiples_copy = valuation_multiples
valuation_multiples = valuation_multiples.rename(columns = {'ebitda':'ev/ebitda', 'ebit':'ev/ebit', 'revenue':'ev/revenue'})
valuation_multiples ['ev/ebitda'] = fundamentals['ev'] / valuation_multiples_copy['ebitda']
valuation_multiples ['ev/ebit'] = fundamentals['ev'] / valuation_multiples_copy['ebit']
valuation_multiples ['ev/revenue']  = fundamentals['ev'] / valuation_multiples_copy['revenue']


In [ ]:
multiples = pd.DataFrame(columns = ['pe', 'ev/ebitda', 'ev/ebit', 'ev/revenue'], index = ['average', 'avg/tgt'] )
multiples.iloc[:, 0] = range(0,2)
comps_df = valuation_multiples[:len(ticker)-1]
tgt_df = valuation_multiples.iloc[-1]
multiples.iloc[0] = comps_df.sum() / (len(ticker))
multiples.iloc[1] = tgt_df / multiples.iloc[0]


In [141]:
# CALCULATING THE INTRINSIC VALUE
iex_api_key = 'sk_29735f4ddf4a47efb27623b229dda54a'
p = "https://cloud.iexapis.com/stable/stock/"+tgt_ticker+"/price?token="+iex_api_key
raw_price = requests.get(p)
price = raw_price.json()  
price_diff = price / multiples.iloc[1]
intrinsic_price = round((sum(price_diff) / 4), 2)

The intrinsic value of Apple : 184.9
